In [1]:
import random
import csv


class Unit(object):
    def __init__(self, id_, owner, type_, real_pop, landscape):
        self.id = id_
        self.owner = owner
        self.type = type_
        self.base_pop = 12000
        self.real_pop = int(real_pop)
        self.landscape = landscape
        
        if self.type == 'INF':
            self.protect = 600
            self.armor = 0
            self.attack = 1200
            self.penetration = 100
            self.dV = 50
        elif self.type == 'MECH':
            self.protect = 800
            self.armor = 200
            self.attack = 2000
            self.penetration = 250
            self.dV = 250
        elif self.type == 'TANK':
            self.protect = 1200
            self.armor = 600
            self.attack = 3500
            self.penetration = 400
            self.dV = 250
        elif self.type == 'SHOCK':
            self.protect = 1000
            self.armor = 450
            self.attack = 5000
            self.penetration = 450 
            self.dV = 200
        elif self.type == 'TO':
            self.protect = 600
            self.armor = 50
            self.attack = 1200
            self.penetration = 250
            self.dV = 50
            
        elif self.type == 'FLEET':
            self.base_pop = 32000
            self.protect = 8000
            self.armor = 2400
            self.attack = 12000
            self.penetration = 2000
            self.dV = 2
        elif self.type == 'TRANS':
            self.base_pop = 32000
            self.protect = 1000
            self.armor = 0
            self.attack = 0
            self.penetration = 0
            self.dV = 2
        else:
            raise LogicalError('Error. Type of unit is not exist.')
        self.landscape_establish()
    
    
    def landscape_establish(self):
        if self.landscape == 'MOUNT':
            landscape_dict = {'INF': [1.5, 0.9, 0.8], 
                              'MECH': [1.4, 0.7, 0.5], 
                              'TANK': [1.1, 0.4, 0.5],
                              'SHOCK': [1.25, 0.6, 0.3],
                              'TO': [1.5, 1.3, 1]}
            
        elif self.landscape == 'FOREST':
            landscape_dict = {'INF': [1.2, 0.95, 0.9], 
                              'MECH': [1.05, 0.8, 0.75], 
                              'TANK': [1.05, 0.7, 0.75],
                              'SHOCK': [1.1, 0.75, 0.8], 
                              'TO': [1.3, 1.15, 1]}
        
        elif self.landscape == 'FLAT':
            landscape_dict = {'INF': [1, 1, 1], 
                              'MECH': [1, 1.05, 1], 
                              'TANK': [1, 1.05, 1],
                              'SHOCK': [1, 1, 1], 
                              'TO': [0.9, 0.9, 1]}
        elif self.landscape == 'SEA':
            landscape_dict = {'FLEET': [1, 1, 1],
                              'TRANS': [1, 1, 1]}
            
        self.k1 = landscape_dict[self.type][0]
        self.k2 = landscape_dict[self.type][1]
        self.k3 = landscape_dict[self.type][2]        
        
         
    def k_update(self, k1, k2, k3):
        self.k1 += k1 # protect (k2)
        self.k2 += k2 # attack (k1)
        self.k3 += k3 #dV
        
            
    def spec_update(self):
        indicator = self.real_pop / self.base_pop
        self.protect = round(self.protect * indicator)
        self.attack = round(self.attack * indicator)
        
        
    def landscape_change(self, new_landscape):
        self.landscape = new_landscape
        self.spec_update()
        self.landscape_establish()
        
    
    def health(self):
        if self.real_pop != 12000:
            if 12000 - self.real_pop >= 1200:
                self.real_pop = self.real_pop + 1200
            else:
                self.real_pop = 12000 
                               
                
    def info(self):
        return self.owner, self.id, self.real_pop, self.attack, self.penetration, self.protect, self.armor, self.k2, self.k1
    
    def attacked_by_fleet(self, enemy_fleet):
        n = round(random.uniform(-1, 1), 5)
        propability_factor = 1 + (n / 4)
        dX1 = round(1200 * propability_factor)
        old_pop = self.real_pop
        self.real_pop = self.real_pop - dX1
        
        print(f'////////// ЛОГ УДАРА //////////')
        print(f'// Фактор случайности: {propability_factor}')
        print(f'// Итоговый урон: {dX1}')
        print(f'// Численность до удара: {old_pop}')
        print(f'// Численность после удара: {self.real_pop}')
        print(f'// Ударил флот: {enemy_fleet.id}; государства {enemy_fleet.owner}')
    
    
    def attack_enemy(self, enemy: object):
        n = round(random.uniform(-1, 1), 5)
        propability_factor = 1 + (n / 4)
        dX1 = round((1 + (n / 4)) * (self.k2 * self.attack - (enemy.armor - self.penetration) - enemy.k1 * enemy.protect))
        w = round(random.uniform(-1, 1), 5)
        enemy_propability_factor = (1 + (w / 4))
        enemy_dX1 = round((1 + (w / 4)) * (enemy.k2 * enemy.attack - (self.armor - enemy.penetration) - self.k1 * self.protect))
        
        if dX1 < 0:
            dX1 = 0
            print('Нулевой урон по врагу.')

        if dX1 >= 4000:
            retreat = True
        else:
            retreat = False
    
        if enemy_dX1 < 0:
            enemy_dX1 = 0
            print('Нулевой урон врагам.')
            
        print(f'///////////// ЛОГ БОЯ /////////////')
        print(f'// Дивизия должна отступить? {retreat}')
        old_pop = self.real_pop
        old_enemy_pop = enemy.real_pop        
        self.real_pop = self.real_pop - enemy_dX1
        self.spec_update()        
        enemy.real_pop = enemy.real_pop - dX1
        enemy.spec_update()       
        print(f'// Численность атакующего после атаки: {self.real_pop}')
        print(f'// Численность защищающегося после атаки: {enemy.real_pop}')
        

def unit_reader():
    units = []
    path = 'Units.csv'
    with open(path, 'r', encoding='utf=8') as file:
        reader = csv.DictReader(file, delimiter=';')
        for row in reader:
            units.append(Unit(*row.values()))
    return units

def unit_data_creator(units):
    data = []
    result_data = []
    data.append(['ID', 'OWNER', 'TYPE', 'REAL_POP', 'LANDSCAPE'])
    
    for index in range(len(units)):  
        data.append([units[index].id, units[index].owner, units[index].type, units[index].real_pop, units[index].landscape])
    fieldnames = data[0]
    for values in data[1:]:
        result_data.append(dict(zip(fieldnames, values)))
    return fieldnames, result_data

def unit_writer(fieldnames, data):
    path = 'Units.csv'
    with open(path, 'w', newline='', encoding='utf=8') as output:
        writer = csv.DictWriter(output, delimiter=';', fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)
            
def war(units_dict):  
    while True:
        attacker_id = int(input('Атакует дивизия с ID: '))
        defender_id = int(input('Защищается дивизия с ID: '))
        accept = input('Вы подтверждаете выбор? ')
    
        accept_list = ['+', 'lf', 'да', 'LF', 'lF', 'Lf', 'Да', 'ДА']
    
        if accept in accept_list:
            attacker = units_dict[attacker_id]
            defender = units_dict[defender_id]
            attacker.attack_enemy(defender)
        else:
            print('Выбор отменён')
        choise = input('\n\nПродолжить? ')
        
        if choise in accept_list:
            continue
        else:
            break
            
        
def health_all(units):
    for unit in units:
        unit.health()
            

units = unit_reader()            
ids = [int(unit.id) for unit in units]
units_dict = {int(id_): unit for (id_, unit) in zip(ids, units)}

war(units_dict)

unit_writer(*unit_data_creator(units))

Атакует дивизия с ID: 101
Защищается дивизия с ID: 404
Вы подтверждаете выбор? +
///////////// ЛОГ БОЯ /////////////
// Дивизия должна отступить? False
// Численность атакующего после атаки: 10913
// Численность защищающегося после атаки: 11643


Продолжить? -


In [ ]:
print('// ')
        print(f'// Атака атакующего: {self.attack}')
        print(f'// Бронепробитие атакующего: {self.penetration}')
        print(f'// Параметр k1 атакующего: {self.k2}')
        print('// ')
        print(f'// Атака защищающегося: {enemy.attack}')
        print(f'// Бронепробитие защищающегося: {enemy.penetration}')
        print(f'// Защита защищающегося: {enemy.protect}')
        print(f'// Броня защищающегося: {enemy.armor}')
        print(f'// Параметр k2 защищающегося: {self.k1}')
        print('// ')
        print(f'// Результат бронепробития: {enemy.armor - self.penetration}')
        print(f'// Результат вражеского бронепробития: {self.armor - self.penetration}')

In [ ]:
class Log(object):
    def __init__(self, time, )


def log_reader():
    path = 'log.csv'
    logs = []
    with open(path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimeter=';')
        for log in reader:
            logs.append([log])
    return logs

def logs_writer():
    path = 'log.csv'
    
    data = ['Time', 'Attacker', '']

In [ ]:

def unit_reader():
    units = []
    path = 'Units.csv'
    with open(path, 'r', encoding='utf=8') as file:
        reader = csv.DictReader(file, delimiter=';')
        for row in reader:
            units.append([row])
    return units

def unit_data_creator(units):
    data = []
    result_data = []
    data.append(['ID', 'OWNER', 'TYPE', 'REAL_POP', 'LANDSCAPE'])
    
    for index in range(len(units)):  
        data.append([units[index].id, units[index].owner, units[index].type, units[index].real_pop, units[index].landscape])
    fieldnames = data[0]
    for values in data[1:]:
        result_data.append(dict(zip(fieldnames, values)))
    return fieldnames, result_data

def unit_writer(fieldnames, data):
    path = 'Units.csv'
    with open(path, 'w', newline='', encoding='utf=8') as output:
        writer = csv.DictWriter(output, delimiter=';', fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)